In [1]:
import pandas as pd

In [2]:
df = pd.read_json("novice/nlp.jsonl", lines=True)

In [3]:
#append '0' in front of anything that starts with 0 bc pandas trolling

In [4]:
df["heading"] = df["heading"].apply(lambda x: str(x) if len(str(x))==3 else '0'+str(x))

In [5]:
df["heading"] = df["heading"].apply(lambda x: str(x) if len(str(x))==3 else '0'+str(x))

In [6]:
df

,key,transcript,tool,heading,target
0,0,"Heading is one five zero, target is green comm...",electromagnetic pulse,150,green commercial aircraft
1,1,"Heading is two six zero, target is black, whit...",surface-to-air missiles,260,"black, white, and yellow commercial aircraft"
2,2,"Heading is one zero five, target is silver, gr...",anti-air artillery,105,"silver, green, and yellow light aircraft"
3,3,"Heading is two niner zero, target is brown and...",electromagnetic pulse,290,brown and blue cargo aircraft
4,4,"Heading is zero one five, target is yellow cam...",EMP,015,yellow camouflage drone
...,...,...,...,...,...
3495,3495,"Heading is zero niner five, target is blue hel...",surface-to-air missiles,095,blue helicopter
3496,3496,"Heading is zero three zero, target is grey, re...",surface-to-air missiles,030,"grey, red, and purple light aircraft"
3497,3497,"Heading is two two five, target is green and g...",surface-to-air missiles,225,green and grey helicopter
3498,3498,"Heading is one eight five, target is red, brow...",surface-to-air missiles,185,"red, brown, and yellow commercial aircraft"


In [7]:
all([len(i)==3 for i in df["heading"]])

True

In [8]:
#map heading back into words

In [9]:
nmap = {'0': 'zero',
 '1': 'one',
 '2': 'two',
 '3': 'three',
 '4': 'four',
 '5': 'five',
 '6': 'six',
 '7': 'seven',
 '8': 'eight',
 '9': 'niner'}

In [10]:
id_2_label = {0:"O",
                  1:"B-TOO",
                  2:"I-TOO",
                  3:"B-HEA",
                  4:"I-HEA",
                  5:"B-TAR",
                  6:"I-TAR"}

label_2_id = {value:key for key, value in id_2_label.items()}
label_list = list(id_2_label.values())
id_2_label, label_2_id, label_list

({0: 'O',
  1: 'B-TOO',
  2: 'I-TOO',
  3: 'B-HEA',
  4: 'I-HEA',
  5: 'B-TAR',
  6: 'I-TAR'},
 {'O': 0,
  'B-TOO': 1,
  'I-TOO': 2,
  'B-HEA': 3,
  'I-HEA': 4,
  'B-TAR': 5,
  'I-TAR': 6},
 ['O', 'B-TOO', 'I-TOO', 'B-HEA', 'I-HEA', 'B-TAR', 'I-TAR'])

In [11]:
df["heading"] = df["heading"].apply(lambda x: ' '.join([nmap[i] for i in str(x)]))

In [12]:
#try keeping the hyphens can try replacing x-x-x with x - x - x, it gets worse

In [13]:
#tag each word in transcript with their respective ner tag

In [14]:
import re
import nltk
nltk.download("punkt")
def create_tag(head_id, inside_id, phrase):
    tags = [inside_id]*(len(tokenize_sentence(phrase)))
    tags[0] = head_id
    return " ".join(tags)
def tokenize_sentence(string):
    return nltk.word_tokenize(split_hyph(string))
def split_hyph(string):
    return string.replace("-", " ")
def labeller(row):
    sentence, tool, heading, target = split_hyph(row["transcript"]), split_hyph(row["tool"]), split_hyph(row["heading"]), split_hyph(row["target"])
    IGNORE = [str(i) for i in id_2_label.keys()]
    tool_tags = create_tag("1", "2", tool)
    #print(tool, tool_tags)
    sentence = re.sub(rf"\b{tool}\b", tool_tags, sentence)
    heading_tags = create_tag("3", "4", heading)
    #print(heading, heading_tags)
    sentence = re.sub(rf"\b{heading}\b", heading_tags, sentence)
    target_tags = create_tag("5", "6", target)
    #print(target, target_tags)
    sentence = re.sub(rf"\b{target}\b", target_tags, sentence)
    #print(tokenize_sentence(sentence))
    return [0 if item not in IGNORE else int(item) for item in tokenize_sentence(sentence)]

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
#create a train ds
df_train = pd.DataFrame()

In [16]:
#can apply some augments here
import nltk
nltk.download("wordnet")
from nltk.corpus import wordnet
import random

def replace_with_synonym(sentence):
    words = sentence.split()
    for i, word in enumerate(words):
        synsets = wordnet.synsets(word)
        if synsets:
            synonym = random.choice(synsets[0].lemmas()).name()
            words[i] = synonym
    return ' '.join(words)
replace_with_synonym("Heading is zero niner five, target is red missile, tool to deploy is surface-to-air missiles.") # in this case we copy over the labels but create another instance of tokenization

[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'header be nothing ennead five, mark be red missile, tool to deploy be surface-to-air missiles.'

In [17]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

def paraphrase_sentence(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)

    for i, (word, tag) in enumerate(tagged_tokens):
        if tag.startswith('N') or tag.startswith('V'):
            synsets = wordnet.synsets(word)
            if synsets:
                synonyms = synsets[0].lemmas()
                if len(synonyms) > 1:
                    for synonym in synonyms:
                        if synonym.name() != word:
                            tagged_tokens[i] = (synonym.name(), tag)
                            break

    return ' '.join(word for word, tag in tagged_tokens)
paraphrase_sentence("Heading is zero niner five, target is red missile, tool to deploy is surface-to-air missiles.")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


'heading is zero nine five , mark is red missile , tool to deploy is surface-to-air missiles .'

In [18]:
df_train["NER_TAGS"] = df.apply(labeller, axis=1)

In [19]:
df_train["TOKENS"] = df["transcript"].apply(tokenize_sentence)

In [20]:
len(df_train["NER_TAGS"][1])

26

In [21]:
len(df_train["TOKENS"][1])

26

In [22]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "albert/albert-base-v2"
#model_checkpoint = "google/fnet-base"
batch_size = 16

In [23]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [24]:
from datasets import Dataset, load_metric

In [25]:
ds = Dataset.from_dict({
    "tokens": df_train["TOKENS"].to_list(),
    "ner_tags": df_train["NER_TAGS"].to_list(),
}).to_iterable_dataset(num_shards=4)

In [26]:
def tokenize_and_align_labels(examples, label_all_tokens = True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    #ner_tags = tokenizer(examples[f"{task}_tags"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [27]:
# tokenized_input = tokenizer(ds[1]["tokens"], is_split_into_words=True)
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
# print(tokens)

In [28]:
# print(tokenized_input.word_ids())

In [29]:
tokenized_datasets = ds.map(tokenize_and_align_labels, batched=True)

In [30]:
iterator = iter(tokenized_datasets)
for i in range(5):
    print(next(iterator))

{'tokens': ['Heading', 'is', 'one', 'five', 'zero', ',', 'target', 'is', 'green', 'commercial', 'aircraft', ',', 'tool', 'to', 'deploy', 'is', 'electromagnetic', 'pulse', '.'], 'ner_tags': [0, 0, 3, 4, 4, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 0, 1, 2, 0], 'input_ids': [2, 4582, 25, 53, 355, 4606, 13, 15, 2935, 25, 647, 1439, 1147, 13, 15, 5607, 20, 17617, 25, 19833, 6181, 13, 9, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 3, 4, 4, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, -100]}
{'tokens': ['Heading', 'is', 'two', 'six', 'zero', ',', 'target', 'is', 'black', ',', 'white', ',', 'and', 'yellow', 'commercial', 'aircraft', ',', 'tool', 'to', 'deploy', 'is', 'surface', 'to', 'air', 'missiles', '.'], 'ner_tags': [0, 0, 3, 4, 4, 0, 0, 0, 5, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0], 'input_ids': [2, 4582, 25, 81, 490, 

In [31]:
label_list

['O', 'B-TOO', 'I-TOO', 'B-HEA', 'I-HEA', 'B-TAR', 'I-TAR']

In [32]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2024-05-19 12:11:30.956112: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-p

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}v2.0",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    dataloader_num_workers = 4,
    report_to=["tensorboard"],
    disable_tqdm=False,
    max_steps=5000,
)

In [34]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [35]:
metric = load_metric("seqeval")

/var/tmp/ipykernel_194345/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [36]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [37]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [38]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,0.000093,1.000000,1.000000,1.000000,1.000000
1,No log,0.000039,1.000000,1.000000,1.000000,1.000000
2,0.021100,0.000022,1.000000,1.000000,1.000000,1.000000
3,0.021100,0.000015,1.000000,1.000000,1.000000,1.000000
4,0.000000,0.000011,1.000000,1.000000,1.000000,1.000000
5,0.000000,0.000008,1.000000,1.000000,1.000000,1.000000
6,0.000000,0.000007,1.000000,1.000000,1.000000,1.000000
7,0.000000,0.000006,1.000000,1.000000,1.000000,1.000000
8,0.000000,0.000005,1.000000,1.000000,1.000000,1.000000
9,0.000000,0.000004,1.000000,1.000000,1.000000,1.000000


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=5000, training_loss=0.0021162212010007353, metrics={'train_runtime': 764.7984, 'train_samples_per_second': 104.603, 'train_steps_per_second': 6.538, 'total_flos': 111161916901308.0, 'train_loss': 0.0021162212010007353, 'epoch': 22.032})

In [39]:
trainer.evaluate()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

{'eval_loss': 1.6338043451469275e-06,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_f1': 1.0,
 'eval_accuracy': 1.0,
 'eval_runtime': 10.3544,
 'eval_samples_per_second': 338.022,
 'eval_steps_per_second': 21.15,
 'epoch': 22.032}

In [40]:
{'eval_loss': 5.253300696494989e-05,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_f1': 1.0,
 'eval_accuracy': 1.0,
 'eval_runtime': 5.8528,
 'eval_samples_per_second': 598.008,
 'eval_steps_per_second': 37.418,
 'epoch': 22.032}
#on original distilbert

{'eval_loss': 5.253300696494989e-05,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_f1': 1.0,
 'eval_accuracy': 1.0,
 'eval_runtime': 5.8528,
 'eval_samples_per_second': 598.008,
 'eval_steps_per_second': 37.418,
 'epoch': 22.032}

In [41]:
predictions, labels, _ = trainer.predict(tokenized_datasets)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

{'HEA': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3500},
 'TAR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3500},
 'TOO': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4150},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [40]:
def combine_labels(entities):
    # Group entities by label
    PUNCTUATIONS = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    groups = {'TOOL': [], 'HEAD': [], 'TARG': []}
    for entity in entities:
        label = entity['entity']
        word = entity['word']
        if label == 'LABEL_1' or label == 'LABEL_2':
            groups['TOOL'].append(word)
        elif label == 'LABEL_3' or label == 'LABEL_4':
            groups['HEAD'].append(word)
        elif label == 'LABEL_5' or label == 'LABEL_6':
            groups['TARG'].append(word)
    
    def join_with_spacing(text_list):
        sentence = ''
        for i, word in enumerate(text_list):
            # If the word contains '##', join it with the previous word
            if word.startswith("##") and i > 0:
                sentence += word[2:]  # Combine the split word without the '##'
            else:
                # Add space before word if it's not the first word and it's not punctuation
                if i != 0 and word not in PUNCTUATIONS:
                    # Add space unless the previous word ends with a hyphen or the current word starts with a hyphen
                    if text_list[i - 1][-1] != '-' and not word.startswith('-'):
                        sentence += ' '
                # Add word to sentence
                sentence += word
        return sentence.strip(PUNCTUATIONS) #clean the ends abit here
        
    return {key:join_with_spacing(value) for key, value in groups.items()}

In [59]:
sentence = "Let's focus on the tall person wearing white, orange, and yellow. We'll use a high-flying drone directed at zero niner five for the mission."
from transformers import pipeline

classifier = pipeline("ner", model="./albert-base-v2-finetuned-nerv2.0/checkpoint-5000", device='cuda:0')
import time
tik = time.time()
data = classifier(sentence)
c= combine_labels(data)
tok = time.time()
print(c, tok-tik)


{'TOOL': '', 'HEAD': '▁zero ▁nine r ▁five', 'TARG': '▁the ▁tall ▁person ▁wearing ▁white, ▁orange, ▁and ▁yellow ▁high-flying ▁drone'} 0.014091968536376953


In [374]:


# Example usage:
text_list = ['we', 'a', 'surface', '-', 'to', '-', 'air', 'missiles']
sentence = join_with_spacing(text_list)
print(sentence)

we a surface-to-air missiles


In [409]:
import pandas as pd

In [410]:
df = pd.read_csv("uvcs/nlp_results.csv")

In [429]:
import ast
for i, item in enumerate(df.iterrows()):
    truth = ast.literal_eval(item[1]["truth"])
    pred = ast.literal_eval(item[1]["prediction"])
    #heading target tool
    if truth["heading"]!=pred["heading"]:
        print(i, "headers", truth["heading"], pred["heading"])

55 headers 005 05
140 headers 005 05
386 headers 005 05
464 headers 005 05
531 headers 005 05
571 headers 005 05
602 headers 005 05
626 headers 005 05
629 headers 005 05
636 headers 005 05
757 headers 005 05
853 headers 005 05
902 headers 005 05
937 headers 005 05
972 headers 005 05
1020 headers 005 05
1117 headers 005 05
1157 headers 005 05
1163 headers 005 05
1242 headers 005 05
1423 headers 005 05
1507 headers 005 05
1601 headers 005 05
1634 headers 005 05
1673 headers 005 05
1757 headers 005 05
1777 headers 005 05
1943 headers 005 05
1962 headers 005 05
2087 headers 005 05
2169 headers 005 05
2181 headers 005 05
2269 headers 005 05
2288 headers 005 05
2403 headers 005 05
2646 headers 005 05
2778 headers 005 05
2910 headers 005 05
2925 headers 005 05
2958 headers 005 05
3017 headers 005 05
3108 headers 005 05
3155 headers 005 05
3217 headers 005 05
3246 headers 005 05
3318 headers 005 05
3384 headers 005 05
3464 headers 005 05
3491 headers 005 05
